Machine Learning and Deep Learning with Tensorflow
=============

Long Short Term Memory (LSTM) RNN model
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [55]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import math
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [56]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [57]:
def read_data(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name))
    f.close()
    
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [58]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [59]:
vocab_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])
print("vocabulary size is %d:" %vocab_size)

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ' or char == None:
        return 0
    else:
        print('Unexpected character: %s' % char)
        return 0

def id2char(dictid):
    if dictid > 0 and dictid < 27:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

vocabulary size is 27:
Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [64]:
def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

In [160]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocab_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [65]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
              return i
    return len(distribution) - 1

def sample(prediction, vSize):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vSize], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution(vSize):
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vSize])
    return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [162]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():

    # Parameters:   
    # input: input gate (ix), forget gate (fx), memory cell (cx), output gate (ox)
    X_ifco = tf.Variable(tf.truncated_normal([vocab_size, 4 * num_nodes], -0.1, 0.1))
    # previous output: input gate (im), forget gate (fm), memory cell (cm), output gate (om)
    M_ifco = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
    # bias: input gate (ib), forget gate (if), memory cell (cb), output gate (ob)
    B_ifco = tf.Variable(tf.zeros([1, 4 * num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocab_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocab_size]))

    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        """
        all_matmul =  tf.matmul(i, X_ifco) + tf.matmul(o, M_ifco) + B_ifco
        input_matmul, forget_matmul, update, output_matmul = tf.split(1, 4, all_matmul)
        all_gates = tf.sigmoid(all_matmul)
        input_gate, forget_gate, _, output_gate = tf.split(1, 4, all_gates)
        
        state = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
          tf.placeholder(tf.float32, shape=[batch_size,vocab_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels)))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)

    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocab_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [167]:
num_steps = 7001
summary_frequency = 100

def graphExec(graph):
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print('Initialized')
        mean_loss = 0
        for step in range(num_steps):

            batches = train_batches.next()
            feed_dict = dict()
            for i in range(num_unrollings + 1):
                feed_dict[train_data[i]] = batches[i]
            _, l, predictions, lr = session.run(
                [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
            mean_loss += l
            if step % summary_frequency == 0:
                if step > 0:
                    mean_loss = mean_loss / summary_frequency
                # The mean loss is an estimate of the loss over the last few batches.
                print(
                    'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
                mean_loss = 0
                labels = np.concatenate(list(batches)[1:])
                print('Minibatch perplexity: %.2f' % float(
                    np.exp(logprob(predictions, labels))))
                if step % (summary_frequency * 10) == 0:
                    # Generate some samples.
                    print('=' * 80)
                    for _ in range(5):
                        feed = sample(random_distribution(vocab_size), vocab_size)
                        sentence = characters(feed)[0]
                        reset_sample_state.run()
                        for _ in range(79):
                            prediction = sample_prediction.eval({sample_input: feed})
                            feed = sample(prediction, vocab_size)
                            sentence += characters(feed)[0]
                        print(sentence)
                    print('=' * 80)
                # Measure validation set perplexity.
                reset_sample_state.run()
                valid_logprob = 0
                for _ in range(valid_size):
                    b = valid_batches.next()
                    predictions = sample_prediction.eval({sample_input: b[0]})
                    valid_logprob = valid_logprob + logprob(predictions, b[1])
                print('Validation set perplexity: %.2f' % float(np.exp(
                    valid_logprob / valid_size)))

In [168]:
%time graphExec(graph)

Initialized
Average loss at step 0: 3.298486 learning rate: 10.000000
Minibatch perplexity: 27.07
usvr  osqy  oirfbqvfnl hchrth iis hefuse aslkvwxrmxn tcvmsgorzmtu yueqjltrmvhrau
bltaozqqrcp yaqeb d qprgvkltojumvbvfbfy hoeejgepzvt o gwdiosueuanhilvetttjbiesal
owiwaylxukcbvg r aspyrpepwzksqxsiskn vrhahlriketowrurjslpantiqezcvnjekzpaira  e 
yawb psgpy lzqauc fz toq  to re rx ecqz la tiwadnvemp eefltante xcjt k wnpeljbpc
gz jkrrgxs xvehw wcnzizty  rgjk ynvnbzsongzyxhucieszegvnic  xnd a dlhktmkfevatir
Validation set perplexity: 19.94
Average loss at step 100: 2.599799 learning rate: 10.000000
Minibatch perplexity: 10.13
Validation set perplexity: 10.81
Average loss at step 200: 2.253537 learning rate: 10.000000
Minibatch perplexity: 7.44
Validation set perplexity: 8.63
Average loss at step 300: 2.099551 learning rate: 10.000000
Minibatch perplexity: 7.26
Validation set perplexity: 8.25
Average loss at step 400: 1.998167 learning rate: 10.000000
Minibatch perplexity: 6.41
Validation set per

---
TODO 1
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [80]:
batch_size = 64
num_unrollings = 10
ngram = 2

class nGramBatchGenerator(object):
    def __init__(self, ngram, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self.ngram = ngram
        self.vocab_size = int(math.pow(vocab_size, ngram))
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)] # vector of the batch_size cursors positions in text
        self._last_batch = self._next_batch()

    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = list()
        for b in range(self._batch_size):
            # getting the ids for ngram chars
            j = 0
            nCharId = 0
            for i in range(self.ngram):
                pos = self._cursor[b] + i
                char = self._text[pos] if pos < self._text_size else ' '
                charId = char2id(char)
                if (charId == 0): continue
                else: 
                    nCharId = nCharId + charId*int(math.pow(vocab_size, ngram-i-1))
                    j = j + 1
            batch.append(nCharId)
            self._cursor[b] = (self._cursor[b] + self.ngram - 1) % self._text_size
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones. """ 
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def nGramCharacters(probabilities, ngram):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [formatCharacter(c, ngram)
          for c in np.argmax(probabilities, 1)]

def formatCharacter(c, ngram): 
    formatString = ''
    args = []
    for i in range(ngram):
        idn = (c//int(math.pow(vocab_size, ngram-i-1))) % vocab_size
        args.append(id2char(idn))
        formatString = formatString + '{%d}'%i
    return formatString.format(*args)


train_batches = nGramBatchGenerator(ngram, train_text, batch_size, num_unrollings)
valid_batches = nGramBatchGenerator(ngram, valid_text, 1, 1)


In [81]:
num_nodes = 64
n_vocab_size = int(math.pow(vocab_size, ngram))

graph = tf.Graph()
with graph.as_default():

    # Parameters:   
    # input: input gate (ix), forget gate (fx), memory cell (cx), output gate (ox)
    X_ifco = tf.Variable(tf.truncated_normal([n_vocab_size, 4 * num_nodes], -0.1, 0.1))
    # previous output: input gate (im), forget gate (fm), memory cell (cm), output gate (om)
    M_ifco = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
    # bias: input gate (ib), forget gate (if), memory cell (cb), output gate (ob)
    B_ifco = tf.Variable(tf.zeros([1, 4 * num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, n_vocab_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([n_vocab_size]))

    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        """
        all_matmul = tf.nn.embedding_lookup(X_ifco, i) + tf.matmul(o, M_ifco) + B_ifco
        input_matmul, forget_matmul, update, output_matmul = tf.split(1, 4, all_matmul)
        all_gates = tf.sigmoid(all_matmul)
        input_gate, forget_gate, _, output_gate = tf.split(1, 4, all_gates)
        
        state = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            tf.placeholder(tf.int64, 
                         shape=[batch_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
          tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits, tf.concat(0, train_labels)))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)

    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int64, shape=[1])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(sample_input, 
                                            saved_sample_output, 
                                            saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [83]:
num_steps = 7001
summary_frequency = 100

def graphExec(graph):
    with tf.Session(graph=graph) as session:
        tf.initialize_all_variables().run()
        print('Initialized')
        mean_loss = 0
        for step in range(num_steps):

            batches = train_batches.next()
            feed_dict = dict()
            for i in range(num_unrollings + 1):
                feed_dict[train_data[i]] = batches[i]
            _, l, predictions, lr = session.run([optimizer, loss, train_prediction, 
                                                 learning_rate], feed_dict=feed_dict)
            mean_loss += l
            if step % summary_frequency == 0:
                if step > 0:
                    mean_loss = mean_loss / summary_frequency
                # The mean loss is an estimate of the loss over the last few batches.
                print(
                    'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
                mean_loss = 0
                labels = np.concatenate(list(batches)[1:])
                # create sparse embedding one-hot encoding
                embeddedLabels = np.zeros(predictions.shape)
                for i, j in enumerate(labels):
                    embeddedLabels[i, j] = 1.0   
                print('Minibatch perplexity: %.2f' % float(
                    np.exp(logprob(predictions, embeddedLabels))))
                if step % (summary_frequency * 10) == 0:
                    # Generate some samples.
                    print('=' * 80)
                    for _ in range(5):
                        feed = sample(random_distribution(n_vocab_size), n_vocab_size)
                        sentence = characters(feed)[0]
                        feed = [np.argmax(feed)]
                        reset_sample_state.run()
                        for _ in range(79):
                            prediction = sample_prediction.eval({sample_input: feed})
                            feed = sample(prediction, n_vocab_size)
                            sentence += characters(feed)[0]
                            feed = [np.argmax(feed)]
                        print(sentence)
                    print('=' * 80)
                # Measure validation set perplexity.
                reset_sample_state.run()
                valid_logprob = 0
                for _ in range(valid_size):
                    b = valid_batches.next()
                    predictions = sample_prediction.eval({sample_input: b[0]})
                    labels = np.zeros((1, n_vocab_size))
                    labels[0, b[1]] = 1.0
                    valid_logprob = valid_logprob + logprob(predictions, labels)
                print('Validation set perplexity: %.2f' % float(np.exp(
                    valid_logprob / valid_size)))

In [84]:
%time graphExec(graph)

Initialized
Average loss at step 0: 6.591617 learning rate: 10.000000
Minibatch perplexity: 728.96
        o                    x         n                                        
             o                                       x                   n      
   v                        m    i                               n              
               nh                                                  q    m       
                                                                                
Validation set perplexity: 668.90
Average loss at step 100: 5.352952 learning rate: 10.000000
Minibatch perplexity: 118.92
Validation set perplexity: 115.87
Average loss at step 200: 4.101295 learning rate: 10.000000
Minibatch perplexity: 26.23
Validation set perplexity: 28.94
Average loss at step 300: 2.896761 learning rate: 10.000000
Minibatch perplexity: 12.17
Validation set perplexity: 13.18
Average loss at step 400: 2.395813 learning rate: 10.000000
Minibatch perplexity: 8.43
Validation

---
TODO 2
---------

Add a GRU cell instead of an LSTM cell.

---

---
TODO 3
---------

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [29]:
def test(*chars):
    for i, char in enumerate(chars):
        print("char is: %s" %char)
        print("iti is: %d" %i)

In [30]:
test('a', 'a', 'b', 'c')

char is: a
iti is: 0
char is: a
iti is: 1
char is: b
iti is: 2
char is: c
iti is: 3


In [199]:
def test(*args):
    formatString = ''
    for i in range(len(args)):
        if formatString == '': formatString = '{%d}'%i
        else: formatString = formatString + ',{%d}'%i
    formatString = '('+ formatString + ')'
    return formatString.format(*args)

In [200]:
print(test(1,2,3))

(1,2,3)
